##### From the file "crawler4piotroski_fscore_selenium"

=> flipping stock codes

In [1]:
# importing packages 
from datetime import datetime
import time
import random

import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# sqlalchemy packages 
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, Numeric, String, DateTime, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship, backref
from sqlalchemy.ext.declarative import declarative_base 

In [2]:
# Headless Chrome
options = webdriver.ChromeOptions()
#options.add_argument('headless')
#options.add_argument('window-size=1920x1080')
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome')

In [3]:
# Start Chrome with Selenium
driver = webdriver.Chrome('/Users/daesikkim/Downloads/chromedriver', chrome_options=options) # chrome_options=options
driver.implicitly_wait(3)

In [5]:
# Get the stock data from DB(postgresql)
db_string = "postgresql://daesik:@localhost/db_piotroski"

engine = create_engine(db_string, echo=True)

In [6]:
Session = sessionmaker(bind=engine)
session = Session()

In [7]:
Base = declarative_base()

In [8]:
# Mapping 
class Stock(Base): 
    __tablename__ = 'stocks'
    
    stock_id = Column(Integer, primary_key=True)
    stock_code = Column(String, unique=True, nullable=False, primary_key=True)
    company = Column(String, index=True, unique=True, nullable=False)
    market_type = Column(Integer)
    m_ics = Column(String) 
    w_ics = Column(String)
    created_on = Column(DateTime, default=datetime.utcnow)
    updated_on = Column(DateTime, default=datetime.utcnow)
    
    base_param = relationship('BaseParam', back_populates='stock')
    bookmarket_param = relationship('BookMarketParam', back_populates='stock')
    
    # def __repr__(self):
    #    return "<Stock ==> id : {0}, code : {1}, company : {2}, market_type : {3}, created : {4}, updated : {5}>".format(self.stock_id, 
    #                                                                                                                     self.stock_code, 
    #                                                                                                                     self.company,
    #                                                                                                                     self.market_type,
    #                                                                                                                     self.created_on,
    #                                                                                                                     self.updated_on,)
    

In [9]:
class BaseParam(Base): 
    __tablename__ = 'base_params'
    
    baseparam_id = Column(Integer, primary_key=True)
    stock_code = Column(String, ForeignKey('stocks.stock_code'))
    date = Column(DateTime)
    price_open = Column(Integer)
    price_close = Column(Integer) 
    price_high = Column(Integer)
    price_low = Column(Integer)
    quant = Column(Integer)
    market_sum = Column(Integer)
    
    stock = relationship("Stock", back_populates="base_param")
    

In [10]:
class BookMarketParam(Base): 
    __tablename__ = 'bookmarket_params'
    
    bookmarketparam_id = Column(Integer, primary_key=True)
    stock_code = Column(String, ForeignKey('stocks.stock_code'))
    listed_stock_cnt = Column(Integer)
    property_total = Column(Integer)
    debt_total = Column(Integer)
    pbr = Column(Integer)
    
    stock = relationship('Stock', back_populates='bookmarket_param')

In [11]:
Base.metadata.create_all(engine)

2017-11-16 00:27:28,134 INFO sqlalchemy.engine.base.Engine select version()
2017-11-16 00:27:28,135 INFO sqlalchemy.engine.base.Engine {}
2017-11-16 00:27:28,140 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-11-16 00:27:28,141 INFO sqlalchemy.engine.base.Engine {}
2017-11-16 00:27:28,145 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-11-16 00:27:28,147 INFO sqlalchemy.engine.base.Engine {}
2017-11-16 00:27:28,152 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-11-16 00:27:28,154 INFO sqlalchemy.engine.base.Engine {}
2017-11-16 00:27:28,157 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-11-16 00:27:28,158 INFO sqlalchemy.engine.base.Engine {}
2017-11-16 00:27:28,162 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

In [12]:
code_n_name = session.query(Stock.stock_code, Stock.company).all()

2017-11-16 00:27:29,406 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-16 00:27:29,410 INFO sqlalchemy.engine.base.Engine SELECT stocks.stock_code AS stocks_stock_code, stocks.company AS stocks_company 
FROM stocks
2017-11-16 00:27:29,412 INFO sqlalchemy.engine.base.Engine {}


In [13]:
code_n_name = dict(code_n_name[0:5]) # Adjust number of codes in the future. For development purpose, codes are limited to 5 

In [14]:
# A function to crawl the industry classification, Market & ICS 
def crawl_ics(code): 
    #pArea > div.wrapper-table > div > table > tbody > tr:nth-child(1) > td > dl > dt:nth-child(3)
    # getting the page source and making the soup! 
    html_ics = driver.page_source 
    soup_ics = BeautifulSoup(html_ics, 'lxml')
    
    # getting valus for ICS 
    m_ics = soup_ics.select('#pArea > div.wrapper-table > div > table > tbody > tr:nth-of-type(1) > td > dl > dt:nth-of-type(3)')
    w_ics = soup_ics.select('#pArea > div.wrapper-table > div > table > tbody > tr:nth-of-type(1) > td > dl > dt:nth-of-type(4)')
    
    m_ics = m_ics[0].string.split(' : ')[-1]
    w_ics = w_ics[0].string.split(' : ')[-1]
    
    ics = {'m_ics': m_ics, 'w_ics': w_ics}
    
    return ics

In [15]:
# A function to crawl a balance sheet
def crawl_balsheet(code):
    
    # click Bal Sheet Tab 
    balsheet_tab = driver.find_element_by_css_selector("#rpt_tab2")
    balsheet_tab.click()
    time.sleep(0.8)
    
    # getting the page source and making the soup! 
    html_balsheet = driver.page_source
    soup_balsheet = BeautifulSoup(html_balsheet, 'lxml')
    
    # getting values from "Balance Sheet" 
    total_asset = soup_balsheet.select('#table-content > table > tbody > tr:nth-of-type(1) > td.num')
    lt_debt = soup_balsheet.select('#table-content > table > tbody > tr:nth-of-type(152) > td.num')
    lt_borrowing = soup_balsheet.select('#table-content > table > tbody > tr:nth-of-type(158) > td.num')
    current_asset = soup_balsheet.select('#table-content > table > tbody > tr:nth-of-type(2) > td.num')
    current_liabilities = soup_balsheet.select('#table-content > table > tbody > tr:nth-of-type(110) > td.num')
    shareholder_equity = soup_balsheet.select('#table-content > table > tbody > tr:nth-of-type(188) > td.num')
    
    # convert string --> float 
    balsheet_vals = list(map(lambda l: [float(i.string.replace(',','')) if i.string != u'\xa0' else float(i.string.replace(u'\xa0','0')) for i in l[0:5]], [total_asset, lt_debt, lt_borrowing, current_asset, current_liabilities, shareholder_equity]))
    
    balsheet_dict = {'total_asset': balsheet_vals[0],
                     'lt_debt': balsheet_vals[1],
                     'lt_borrowing': balsheet_vals[2],
                     'current_asset': balsheet_vals[3],
                     'current_liabilities': balsheet_vals[4],
                     'shareholder_equity': balsheet_vals[5]}
    
    return balsheet_dict 

In [16]:
# A function to crawl an income statement 
def crawl_incomestate(code):
    
    # click Income Statement Tab 
    incomestate_tab = driver.find_element_by_css_selector("#rpt_tab1")
    incomestate_tab.click()
    time.sleep(0.8)
    
    # getting the page source and making the soup
    html_incomestate = driver.page_source
    soup_incomestate = BeautifulSoup(html_incomestate, 'lxml') 
    
    # getting values from "Income Statement" 
    operating_profit = soup_incomestate.select('#table-content > table > tbody > tr:nth-of-type(59) > td.num')
    extra_income = soup_incomestate.select('#table-content > table > tbody > tr:nth-of-type(144) > td.num')
    total_sales = soup_incomestate.select('#table-content > table > tbody > tr:nth-of-type(1) > td.num')
    gross_profit = soup_incomestate.select('#table-content > table > tbody > tr:nth-of-type(26) > td.num')
    cogs = soup_incomestate.select('#table-content > table > tbody > tr:nth-of-type(15) > td.num')
    
    # convert string --> float 
    incomestate_vals = list(map(lambda l: [float(i.string.replace(',','')) if i.string != u'\xa0' else float(i.string.replace(u'\xa0','0')) for i in l[0:5]], [operating_profit, extra_income, total_sales, gross_profit, cogs]))
    
    incomestate_dict = {'operating_profit': incomestate_vals[0],
                        'extra_income': incomestate_vals[1],
                        'total_sales': incomestate_vals[2],
                        'gross_profit': incomestate_vals[3],
                        'cogs': incomestate_vals[4]}
    
    return incomestate_dict

In [17]:
# A function to crawl a cash flow 
def crawl_cashflow(code):
    
    # click Cash Flow tab 
    cashflow_tab = driver.find_element_by_css_selector("#rpt_tab3")
    cashflow_tab.click
    time.sleep(0.8)
    
    # getting the page source and making the soup 
    html_cashflow = driver.page_source 
    soup_cashflow = BeautifulSoup(html_cashflow, 'lxml')
    
    # getting values from "Income Statement" 
    cf_operation = soup_cashflow.select('#table-content > table > tbody > tr:nth-of-type(1) > td.num')
    
    # convert string --> float 
    cashflow_vals = list(map(lambda l: [float(i.string.replace(',','')) if i.string != u'\xa0' else float(i.string.replace(u'\xa0', '0')) for i in l[0:5]], [cf_operation]))
    
    cashflow_dict = {'cf_operation': cashflow_vals[0]}
    
    return cashflow_dict

In [28]:
# A function to transform a list to DF 
def merge_n_convert(code, year, fs_dict): 
    dict_individual = {'code': [code]*5,
                       'year': year}
    
    dict_individual.update(fs_dict)
    
    df_individual = pd.DataFrame(dict_individual, columns = ('code', 'm_ics', 'w_ics', 'year', \
                                    'total_asset', 'lt_debt', 'lt_borrowing', 'current_asset', 'current_liabilities', 'shareholder_equity', \
                                    'operating_profit', 'extra_income', 'total_sales', 'gross_profit', 'cogs', \
                                    'cf_operation', \
                                    'f_roa', 'f_cfo', 'f_droa', 'f_accrual', 'f_accrual', 'f_dlever', 'f_dliquid', 'f_equityoffer', 'f_dmargin', 'f_dturnover', 'f_total'))
    
    return df_individual                              

In [29]:
# Creating f score DF 
year = ['2012', '2013', '2014', '2015', '2016']
fs_dict = {}
df_fscore = pd.DataFrame(columns = ('code', 'm_ics', 'w_ics', 'year', \
                                    'total_asset', 'lt_debt', 'lt_borrowing', 'current_asset', 'current_liabilities', 'shareholder_equity', \
                                    'operating_profit', 'extra_income', 'total_sales', 'gross_profit', 'cogs', \
                                    'cf_operation', \
                                    'f_roa', 'f_cfo', 'f_droa', 'f_accrual', 'f_accrual', 'f_dlever', 'f_dliquid', 'f_equityoffer', 'f_dmargin', 'f_dturnover', 'f_total'))


In [30]:
df_fscore

,code,m_ics,w_ics,year,total_asset,lt_debt,lt_borrowing,current_asset,current_liabilities,shareholder_equity,...,f_cfo,f_droa,f_accrual,f_accrual,f_dlever,f_dliquid,f_equityoffer,f_dmargin,f_dturnover,f_total


In [31]:
# One Big Loop through all company codes
for i in code_n_name.keys(): 
    
    # load the page! 
    driver.get("http://finance.naver.com/item/coinfo.nhn?code={}".format(i))
    time.sleep(random.randrange(2, 8, 1))
    
    # move to the relavent frame
    frame = WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "#coinfo_cp")))
    #driver.switch_to.frame(frame)

    # select to the financial statement tab
    finstate_tab = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#header-menu > div.wrapper-menu > dl > dt:nth-of-type(3) > a")))
    finstate_tab.click()
    
    code = i 
    for d in [crawl_balsheet(i), crawl_incomestate(i), crawl_cashflow(i)]:
        fs_dict.update(d)
    
    df_company = merge_n_convert(i, year, fs_dict)
    
    df_fscore = pd.concat([df_fscore, df_company], ignore_index=True)
    
    print(df_fscore)
    

     code m_ics w_ics  year  total_asset   lt_debt  lt_borrowing  \
0  005930   NaN   NaN  2012    1810715.7  126583.1       35212.6   
1  005930   NaN   NaN  2013    2140750.2  127436.0        9027.2   
2  005930   NaN   NaN  2014    2304229.6  103208.6         239.9   
3  005930   NaN   NaN  2015    2421795.2  126168.1        1936.0   
4  005930   NaN   NaN  2016    2621743.2  145072.0       11791.1   

   current_asset  current_liabilities  shareholder_equity   ...    f_cfo  \
0       872690.2             469330.5           1214802.1   ...      NaN   
1      1107602.7             513154.1           1500160.1   ...      NaN   
2      1151460.3             520139.1           1680881.9   ...      NaN   
3      1248147.3             505029.1           1790598.1   ...      NaN   
4      1414297.0             547041.0           1929630.3   ...      NaN   

   f_droa  f_accrual  f_accrual  f_dlever  f_dliquid f_equityoffer f_dmargin  \
0     NaN        NaN        NaN       NaN        NaN  

      code m_ics w_ics  year  total_asset   lt_debt  lt_borrowing  \
0   005930   NaN   NaN  2012    1810715.7  126583.1       35212.6   
1   005930   NaN   NaN  2013    2140750.2  127436.0        9027.2   
2   005930   NaN   NaN  2014    2304229.6  103208.6         239.9   
3   005930   NaN   NaN  2015    2421795.2  126168.1        1936.0   
4   005930   NaN   NaN  2016    2621743.2  145072.0       11791.1   
5   000660   NaN   NaN  2012     186486.9   44680.7       23018.1   
6   000660   NaN   NaN  2013     207973.0   46522.0       17301.8   
7   000660   NaN   NaN  2014     268832.8   30816.7       12627.7   
8   000660   NaN   NaN  2015     296779.1   34495.1       15120.0   
9   000660   NaN   NaN  2016     322160.3   40316.5       20957.4   
10  005935   NaN   NaN  2012    1810715.7  126583.1       35212.6   
11  005935   NaN   NaN  2013    2140750.2  127436.0        9027.2   
12  005935   NaN   NaN  2014    2304229.6  103208.6         239.9   
13  005935   NaN   NaN  2015    24